# Anthropic Functions

This notebook shows how to use an experimental wrapper around Anthropic that gives it the same API as OpenAI Functions.

In [1]:
from langchain_experimental.llms.anthropic_functions import AnthropicFunctions

/Users/harrisonchase/.pyenv/versions/3.9.1/envs/langchain/lib/python3.9/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.14) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


## Initialize Model

You can initialize this wrapper the same way you'd initialize ChatAnthropic

In [2]:
model = AnthropicFunctions(model='claude-2')

## Passing in functions

You can now pass in functions in a similar way

In [3]:
functions=[
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        },
        "required": ["location"]
      }
    }
  ]

In [5]:
from langchain.schema import HumanMessage

In [6]:
response = model.predict_messages(
    [HumanMessage(content="whats the weater in boston?")], 
    functions=functions
)

In [7]:
response

AIMessage(content=' ', additional_kwargs={'function_call': {'name': 'get_current_weather', 'arguments': '{"location": "Boston, MA", "unit": "fahrenheit"}'}}, example=False)

## Using for extraction

You can now use this for extraction.

In [8]:
from langchain.chains import create_extraction_chain
schema = {
    "properties": {
        "name": {"type": "string"},
        "height": {"type": "integer"},
        "hair_color": {"type": "string"},
    },
    "required": ["name", "height"],
}
inp = """
Alex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blonde.
        """

In [9]:
chain = create_extraction_chain(schema, model)

In [10]:
chain.run(inp)

[{'name': 'Alex', 'height': '5', 'hair_color': 'blonde'},
 {'name': 'Claudia', 'height': '6', 'hair_color': 'brunette'}]

## Using for tagging

You can now use this for tagging

In [11]:
from langchain.chains import create_tagging_chain

In [12]:
schema = {
    "properties": {
        "sentiment": {"type": "string"},
        "aggressiveness": {"type": "integer"},
        "language": {"type": "string"},
    }
}

In [14]:
chain = create_tagging_chain(schema, model)

In [15]:
chain.run("this is really cool")

{'sentiment': 'positive', 'aggressiveness': '0', 'language': 'english'}